# Profit making Apps (App Store and Google Play Markets)
---
## Introduction and goal of the project

Our aim in this project is to find mobile app profiles that are profitable for the __App Store__ and __Google Play__ markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

As our company only build apps that are free to download and install, our main source of profit come from in-app ads. The number of users of our apps is critical to our revenue model - the more users who see and engage with the ads, the better. Our goal is to analyze app store data across Google Play & the App Store to provide insights to our developers, and help them understand the type of apps likely to attract more users.

---
## Reading and exploring the data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead:
- __iOs data set (approximately 7 000 iOS apps from the App Store):__ [link](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)
    - Following columns might be of interest in our analysis:
        - <code>prime_genre</code> - Genre of the app
        - <code>rating_count_tot</code> - Total users who have rated the apps across all versions
        - <code>price</code> - To determine if its a free or a paid app
- __Android data set (approximately 10 000 Android apps from Google Play):__[link](https://www.kaggle.com/lava18/google-play-store-apps)
    - Following columns might be of interest in our analysis:
        - <code>Category</code> - Higher level category of the app
        - <code>Genres</code> - Genre of the app
        - <code>Installs</code> - Number of users who have installed the app
        - <code>Type</code> - To determine if its a free or a paid app

We'll start by opening and exploring these two data sets.

In [1]:
from csv import reader

# open Apple Store data set
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios_data = list(read_file)
ios_header = ios_data[0]
ios = ios_data[1:]

#open Google Play data set
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_data = list(read_file)
android_header = android_data[0]
android = android_data[1:]

To make exploration easier, we created a function that can be repeatedly used to print rows in a readable way together with information about the number of rows and columns in the data set.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

#### App Store data

In [3]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, rows_and_columns = True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


#### Google Play data

In [4]:
print(android_header)
print('\n')
explore_data(android, 0, 3, rows_and_columns = True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


---
## Data Cleaning

Before beginning our analysis, we need to make sure the data we analyze is accurate. We will focus on the following:
- Removing inaccurate data
- Removing duplicate entries
- Removing Non-English apps
- Isolating free apps

### Removing inaccurate data

The [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) for the Google Play Store describes an error for the row: 10472. Let's check the given error and compare the entry with a correct row.

In [5]:
print(android[10472])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


We see that there has been a column shift in the row 10472 when we compare the header row with the given row as: <code>Catergory</code> does not have a numerical value of ('1.9'). This row needs to be deleted.

In [6]:
print("Before deletion:", len(android))
del android[10472]
print("After deletion:", len(android))

Before deletion: 10841
After deletion: 10840


### Removing duplicate entries

Looking through the [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion) for the Google Play Store, there is mention of duplicate entries for applications. For instance, Instagram has four entries:

In [7]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


#### Duplicates in the Android data set

In [8]:
unique_rows = []
duplicate_rows = []

for app in android:
    name = app[0]
    if name in unique_rows:
        duplicate_rows.append(name)
    else:
        unique_rows.append(name)
        
print('Number of unique rows: ', len(unique_rows))
print('Number of duplicate rows: ', len(duplicate_rows))

Number of unique rows:  9659
Number of duplicate rows:  1181


We see __1181 duplicate rows__ in the dataset. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

To do that, we will:
- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

In [9]:
reviews_max = {}
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews

In [10]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Using the above dictionary we build the final list of clean android app dataset - <code>android_clean</code>. To build the clean list:
- We check each app row and its reviews if it matches the max reviews in our <code>reviews_max</code> dictionary
- Also to make sure we add the app only once (if we had more entries with the same max reviews), we use a supplementary list called <code>already_added</code>

In [11]:
android_clean = []
already_added = []
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
        

Now let's quickly explore the new data set, and confirm that the number of rows is __9,659__.

In [12]:
explore_data(android_clean, 0, 3, rows_and_columns=True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


#### Duplicates in the iOS data set

In [13]:
unique_rows_ios = []
duplicate_rows_ios = []

for app in ios:
    name = app[0]
    if name in unique_rows_ios:
        duplicate_rows_ios.append(name)
    else:
        unique_rows_ios.append(name)
        
print('Number of unique rows: ', len(unique_rows_ios))
print('Number of duplicate rows: ', len(duplicate_rows_ios))

Number of unique rows:  7197
Number of duplicate rows:  0


### Removing non-english apps

If we explore the datasets enough, we will notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets.

In [14]:
print('iOS Apps')
print(ios[813])
print(ios[6731])
print('\n')
print('Android Apps')
print(android_clean[4412])
print(android_clean[7940])

iOS Apps
['445375097', '爱奇艺PPS -《欢乐颂2》电视剧热播', '224617472', 'USD', '0.0', '14844', '0', '4.0', '0.0', '6.3.3', '17+', 'Entertainment', '38', '5', '3', '1']
['1120021683', '【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜', '77551616', 'USD', '0.0', '0', '0', '0.0', '0.0', '1.3', '12+', 'Games', '38', '0', '1', '1']


Android Apps
['中国語 AQリスニング', 'FAMILY', 'NaN', '21', '17M', '5,000+', 'Free', '0', 'Everyone', 'Education', 'June 22, 2016', '2.4.0', '4.0 and up']
['لعبة تقدر تربح DZ', 'FAMILY', '4.2', '238', '6.8M', '10,000+', 'Free', '0', 'Everyone', 'Education', 'November 18, 2016', '6.0.0.0', '4.1 and up']


We're not interested in keeping these kind of apps, so we'll remove them. All the characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

To ensure that we do not ignore apps that contain special characters such as ™ or 😜 in their app names (as these have ASCII value > 127), we are going to establish a rule that up to 3 characters with ASCII value greater than 127 is allowed.

In [15]:
def is_english(string):
    non_ascii = 0
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
        
    if non_ascii > 3:
        return False
    else:
        return True


To ensure that this function works as intended, we will use certain app names to determine the result.

In [16]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


Let's now use this new function on our AppStore (ios) and Google PlayStore (android_clean) data sets.

In [17]:
ios_english = []

for app in ios:
    name = app[1]
    if is_english(name) == True:
        ios_english.append(app)
        
print('Number of English apps ios: ', len(ios_english))    

Number of English apps ios:  6183


In [18]:
android_english = []

for app in android_clean:
    name = app[0]
    if is_english(name) == True:
        android_english.append(app)
        
print('Number of English apps android: ', len(android_english))  

Number of English apps android:  9614


### Isolating free apps

We are looking to develop a free app and maximise our ad-revenue and hence for our analysis we only need the free apps to be considered for our profiling. Below, we isolate the free apps for both our data sets.

In [19]:
ios_en_free = []
android_en_free = []

for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_en_free.append(app)
        
for app in android_english:
    price = app[7]
    if price == '0':
        android_en_free.append(app)
        

print('Number of free english iOS apps:', len(ios_en_free))
print('Number of free english Android apps:', len(android_en_free))

Number of free english iOS apps: 3222
Number of free english Android apps: 8864


---
## Data analysis

As we mentioned in the introduction, our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets.

### Most common apps by genre

We need two functions to produce usable information from the dataset:
- Function to produce a frequency table for the given column showing percentages
- Function to show the percentages in a decscending order

In [20]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for app in dataset:
        total += 1
        column = app[index]
        if column in table:
            table[column] += 1
        else:
            table[column] = 1
            
    table_percentages = {}        
    for key in table:
        percentage = table[key]/total * 100
        table_percentages[key] = percentage
        
    return table_percentages

In [21]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

#### Most common apps by genre: App Store

We start our analysis by checking the <code>prime_genre</code> column of the App Store dataset.

In [22]:
display_table(ios_en_free, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most common genre is __Games (58%)__ by a wide margin. The second most common is __Entertainment (8%)__. Based on the top 5 results, most of the apps designed are for entertainment purposes. Practical purposes apps (education, shopping, utilities, productiy and lifestyle) lag behind enterainment. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users.

#### Most common apps by genre: Google Play Store

Here it is a little complicated that we have two columns (<code>Genres</code> and <code>Category</code>) for the purpose of categorisation. Now if we look at the <code>Genres</code> column we see that data is too granular to be able to stay categorically at a high-level similar to the AppStore.

So for our sub-sequent analysis we are going to consider the <code>Category</code> column.

In [23]:
display_table(android_en_free, -4) # Genres column

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [24]:
display_table(android_en_free, 1) # Category column

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The free English apps in Google Play Store differ in comparison to the App Store. Here we have __FAMILY (19%)__ at the top of the list. However, GAME __(9.72%)__ is a close second. Based on the Category column, pracitcal purpose apps such as __TOOLS__, __BUSINES__, __LIFESTYLE__, & __PRODUCTIVITY__ account for __(21%)__.

### Most Popular Apps by Genre

We're going to examine the kind of apps with the most users. One method is to calculate the average number of downloads for each app genre. For the Google Play data set, we can find this information in the <code>Installs</code> column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the <code>rating_count_column</code> app.

#### Most Popular Apps by Genre: App Store

Let's start with calculating the average number of user ratings per app genre on the App Store. To do that, we'll need to do the following:
- Isolate the apps of each genre
- Add up the user ratings for the apps of that genre
- Divide the sum by the number of apps belonging to that genre (not by the total number of apps)

In [25]:
prime_genre_freq = freq_table(ios_en_free, -5)
for genre in prime_genre_freq:
    total = 0 # number of ratings
    len_genre = 0 # number of apps specific to each genre
    
    for app in ios_en_free:
        genre_app = app[-5]
        if genre_app == genre:
            ratings = float(app[5])
            total += ratings
            len_genre += 1
            
    average_ratings = total / len_genre
    print(genre, ':', average_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


__Social Networking__ genre have an average of __71,548__ ratings per app. However, we would have a lot of rivals like Facebook and Pinterest if we stand here.

Let's look a bit closer at the __Reference__ genre with an average of __74,942__ ratings per app.

In [26]:
for app in ios_en_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Even though the Bible and Dictionary apps skew up the average rating we can see some potential here. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea also seems to fit well with the fact that the Google Play Store is dominated by practical apps and in the end we are looking for an app that can be successfull on both platforms.

#### Most Popular Apps by Genre: Google Play Store

In the Google Play Store dataset, the <code>Installs</code> column has a range base such as 1,000+, 10,000+ but for our analysis we will consider these are hard numbers (By replacing "," and "+") as we are going to employ the same technique across the dataset it should not cause any error in judgment.

In [27]:
category_freq_table = freq_table(android_en_free, 1)
table_display = []

for category in category_freq_table:
    total = 0
    len_category = 0
    
    for app in android_en_free:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            n_installs = float(n_installs)
            total += n_installs
            len_category += 1
            
    average_android = total / len_category
    value_key_tpl = (average_android, category)
    table_display.append(value_key_tpl)

for entry in sorted(table_display, reverse = True):
    print(entry[1],':',entry[0])

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

We see that, on average, __communication__ apps have most installs. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [28]:
for app in android_en_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

We see the same pattern for the __video players__ category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

The __game__ genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The __books and reference__ genre looks fairly popular as well, with an average number of installs of __8,767,811__. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [29]:
for app in android_en_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [30]:
for app in android_en_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

---
## Conclusion

The goal of the project was to find an app profile, in Google Play Store and the App Store, with high user engagement to recommend to our developers. Based on our analysis, we've concluded to build an app based on a popular book. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.
